# 调用多个厂商的大模型 API

注意: 

本节代码使用的是中转服务，接口均为 OpenAI API 兼容类型，只需要改模型名称即可。

更详细说明: https://positive-octagon-a48.notion.site/LLM-engineering-Course-Week-2-OpenAI-Claude-Gemini-API-19c56546c5d280f3905ac2b6585d3657?pvs=4

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')

In [ ]:
# 所有模型均使用 OpenAI 类型的接口调用
openai = OpenAI(api_key=api_key, base_url=base_url)

In [ ]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [ ]:
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
]

In [ ]:
# gpt-4o-mini

response = openai.chat.completions.create(
    model='gpt-4o-mini', messages=messages)
print(response.choices[0].message.content)

In [ ]:
# Gemini 2.0 flash

response = openai.chat.completions.create(
    model='gemini-2.0-flash', messages=messages)
print(response.choices[0].message.content)

In [ ]:
# DeepSeek V3

response = openai.chat.completions.create(
    model='deepseek-chat', messages=messages)
print(response.choices[0].message.content)

## 两个模型辩论

In [ ]:
gpt_model = 'gpt-4o-mini'
gemini_model = 'gemini-2.0-flash'

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

gemini_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
gemini_messages = ["Hi"]

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, gemini in zip(gpt_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": gemini})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
call_gpt()

In [ ]:
def call_gemini():
    messages = [{"role": "system", "content": gemini_system}]
    for gpt, gemini in zip(gpt_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": gemini})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    completion = openai.chat.completions.create(
        model=gemini_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
call_gemini()

In [ ]:
call_gpt()

In [ ]:
gpt_messages = ["Hi there"]
gemini_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    gemini_next = call_gemini()
    print(f"Gemini:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)


In [ ]:
# 上面的代码不是很直观，让 AI 重构了一下
# 核心思想是每个 AI 回复的内容存在各自的 list 中
# 每个 AI 的最后一条回复是另一个 AI 的最新的"user"输入

# 模型配置
GPT_MODEL = 'gpt-4o-mini'
GEMINI_MODEL = 'gemini-1.5-flash' # 2.0 flash 请求过于频繁会返回空...

# 系统提示词配置
GPT_SYSTEM_PROMPT = """你是一个非常喜欢争论的聊天机器人；你不同意对话中的任何事情，并且以一种讽刺的方式挑战一切。"""
GEMINI_SYSTEM_PROMPT = """你是一个非常有礼貌、客气的聊天机器人。 你尽量同意别人说的一切，或者找到共同点。 如果对方喜欢争论，你会尽量安抚他们并继续聊天。"""

# 初始化消息列表
GPT_MESSAGES = [
    {"role": "system", "content": GPT_SYSTEM_PROMPT},
    {"role": "assistant", "content": "你好呀"}
]

GEMINI_MESSAGES = [
    {"role": "system", "content": GEMINI_SYSTEM_PROMPT},
    {"role": "assistant", "content": "你好"}
]


# 定义调用 OpenAI API 的函数
def call_openai_api(model, messages):
    """调用 OpenAI API 获取模型回复."""
    completion = openai.chat.completions.create(
        model=model,
        messages=messages
    )
    return completion.choices[0].message.content


# 定义 GPT 模型的函数
def get_gpt_response():
    """获取 GPT 模型的回复."""
    response = call_openai_api(GPT_MODEL, GPT_MESSAGES)
    GPT_MESSAGES.append({"role": "user", "content": GEMINI_MESSAGES[-1]['content']})  # 添加 Gemini 的上一条消息
    GPT_MESSAGES.append({"role": "assistant", "content": response})  # 添加 GPT 的回复
    return response


# 定义 Gemini 模型的函数
def get_gemini_response():
    """获取 Gemini 模型的回复."""
    response = call_openai_api(GEMINI_MODEL, GEMINI_MESSAGES)
    GEMINI_MESSAGES.append({"role": "user", "content": GPT_MESSAGES[-1]['content']})  # 添加 GPT 的上一条消息
    GEMINI_MESSAGES.append({"role": "assistant", "content": response})  # 添加 Gemini 的回复
    return response


# 主循环
print("GPT: 你好呀\n")
print("Gemini: 你好\n")

for _ in range(5):
    # 获取 GPT 的回复
    gpt_response = get_gpt_response()
    print(f"GPT: {gpt_response}\n")

    # 获取 Gemini 的回复
    gemini_response = get_gemini_response()
    print(f"Gemini: {gemini_response}\n")
